In [ ]:
"""2D Multi-Object Search (MOS) + manipulation Task.
Uses the domain, models, and agent/environment
to actually define the POMDP problem for multi-object search.
Then, solve it using POUCT or POMCP"""
import pomdp_py
from pomdp_problems.rearrange_pomdp.env.env import *
from pomdp_problems.rearrange_pomdp.env.visual import *
from pomdp_problems.rearrange_pomdp.agent.agent import *
from pomdp_problems.rearrange_pomdp.example_worlds import *
from pomdp_problems.rearrange_pomdp.domain.observation import *
from pomdp_problems.rearrange_pomdp.models.components.grid_map import *
import argparse
import time
import random
from icecream import ic
from pomdp_problems.rearrange_pomdp.manip_problem import ManipOOPOMDP, belief_update,belief_update_manip

In [ ]:
def print_stats(problem):
    ic (problem)
    robot_id = problem.agent.robot_id
    ic (set(problem.env.state.object_states[robot_id].objects_found))
    ic (problem.env.target_objects)
    ic (len(problem.env.target_objects))

def print_info(problem,planner, robot_id, real_action,real_observation,reward, 
               step,_total_reward,_find_actions_count,_pick_actions_count):
    print("==== Step %d ====" % (step+1))
    print("Action: %s" % str(real_action))
    print("Observation: %s" % str(real_observation))
    print("Reward: %s" % str(reward))
    print("Reward (Cumulative): %s" % str(_total_reward))
    print("Find Actions Count: %d" %  _find_actions_count)
    print("Pick Actions Count: %d" % _pick_actions_count)
    ic (problem.env.state.object_states[robot_id]['objects_picked'])

    if isinstance(planner, pomdp_py.POUCT):
        print("__num_sims__: %d" % planner.last_num_sims)

def update_action_counts(real_action, _find_actions_count,_pick_actions_count):

    if isinstance(real_action, FindAction):
        _find_actions_count += 1
    if isinstance(real_action,PickAction):
        _pick_actions_count += 1

    return _find_actions_count, _pick_actions_count

In [ ]:
def visualize(viz, problem, robot_id, real_action, real_observation):
    # This is used to show the sensing range; Not sampled
    # according to observation model.
    robot_pose = problem.env.state.object_states[robot_id].pose
    viz_observation = MosOOObservation({})
    if isinstance(real_action, LookAction) or isinstance(real_action, FindAction):
        viz_observation = \
            problem.env.sensors[robot_id].observe(robot_pose,
                                                    problem.env.state)
    viz.update(robot_id,
                real_action,
                real_observation,
                viz_observation,
                problem.agent.cur_belief)
    viz.on_loop()
    viz.on_render()

In [ ]:
def termination_condition_check(problem, robot_id, _find_actions_count,
                                 _pick_actions_count, _time_used,max_time):

    if set(problem.env.state.object_states[robot_id].objects_found)\
        == problem.env.target_objects  and \
        problem.env.state.object_states[robot_id].objects_picked \
        == len(problem.env.target_objects):
        ic (problem.env.state.object_states[robot_id].objects_found)
        ic (problem.env.state.object_states[robot_id].objects_picked)
        ic (len(problem.env.target_objects))
        print("Done!")
        #break
        return True
    if _find_actions_count >= len(problem.env.target_objects) \
        and _pick_actions_count >= len(problem.env.target_objects):
        print("FindAction limit reached, pick actions currently not checked limit limit reached.")
        return True
    if _time_used > max_time:
        print("Maximum time reached.")
        return True
    return False

In [ ]:

### Solve the problem with POUCT/POMCP planner ###
### This is the main online POMDP solver logic ###
def solve_init(problem,
          max_depth=10,  # planning horizon
          discount_factor=0.99,
          planning_time=1.,       # amount of time (s) to plan each step
          exploration_const=1000, # exploration constant
          visualize=True,
          max_time=120,  # maximum amount of time allowed to solve the problem
          max_steps=500): # maximum number of planning steps the agent can take.
    """
    This function terminates when:
    - maximum time (max_time) reached; This time includes planning and updates
    - agent has planned `max_steps` number of steps
    - agent has taken n FindAction(s) where n = number of target objects.

    Args:
        visualize (bool) if True, show the pygame visualization.
    """

    random_objid = random.sample(problem.env.target_objects, 1)[0]
    random_object_belief = problem.agent.belief.object_beliefs[random_objid]
    if isinstance(random_object_belief, pomdp_py.Histogram):
        # Use POUCT
        planner = pomdp_py.POUCT(max_depth=max_depth,
                                 discount_factor=discount_factor,
                                 planning_time=planning_time,
                                 exploration_const=exploration_const,
                                 rollout_policy=problem.agent.policy_model)  # Random by default
    elif isinstance(random_object_belief, pomdp_py.Particles):
        # Use POMCP
        planner = pomdp_py.POMCP(max_depth=max_depth,
                                 discount_factor=discount_factor,
                                 planning_time=planning_time,
                                 exploration_const=exploration_const,
                                 rollout_policy=problem.agent.policy_model)  # Random by default
    else:
        raise ValueError("Unsupported object belief type %s" % str(type(random_object_belief)))

    robot_id = problem.agent.robot_id
    viz = None
    if visualize:
        viz = MosViz(problem.env, controllable=False)  # controllable=False means no keyboard control.
        if viz.on_init() == False:
            raise Exception("Environment failed to initialize")
        viz.update(robot_id,
                   None,
                   None,
                   None,
                   problem.agent.cur_belief)
        viz.on_render()

    return planner, robot_id, viz

    

In [ ]:
seed = 10
grid_map, robot_char = random_world(10, 10, 5, 10,seed=seed)
#ic (grid_map)
#exit()
laserstr = make_laser_sensor(90, (1, 4), 0.5, False)
proxstr = make_proximity_sensor(4, False)
problem = ManipOOPOMDP(robot_char,  # r is the robot character
                        sigma=0.05,  # observation model parameter
                        epsilon=0.95, # observation model parameter
                        grid_map=grid_map,
                        sensors={robot_char: proxstr},
                        prior="uniform",
                        agent_has_map=True)
max_steps = 5000
max_time = 120
visualize = False

planner, robot_id, viz = solve_init(problem,
        max_depth=10,
        discount_factor=0.99,
        planning_time=1.,
        exploration_const=1000,
        visualize=visualize,
        max_time=120,
        max_steps=5000)

In [ ]:
_time_used = 0
_find_actions_count = 0
_pick_actions_count = 0
_total_reward = 0  # total, undiscounted reward
for i in range(max_steps):
    # Plan action
    _start = time.time()
    real_action = planner.plan(problem.agent)
    _time_used += time.time() - _start
    if _time_used > max_time:
        break  # no more time to update.

    # Execute action
    reward = problem.env.state_transition(real_action, execute=True,
                                            robot_id=robot_id)

    # Receive observation
    _start = time.time()
    real_observation = \
        problem.env.provide_observation(problem.agent.observation_model, real_action)

    # Updates
    problem.agent.clear_history()  # truncate history
    problem.agent.update_history(real_action, real_observation)
    belief_update_manip(problem.agent, real_action, real_observation,
                    problem.env.state.object_states[robot_id],
                    planner)
    _time_used += time.time() - _start

    _total_reward += reward
    # Info and render
    _find_actions_count, _pick_actions_count = update_action_counts(real_action, _find_actions_count,_pick_actions_count)

    print_info(problem, planner,robot_id, real_action,real_observation,reward, i
                ,_total_reward, _find_actions_count,_pick_actions_count)

    if visualize:
        visualize(viz, problem, robot_id, real_action, real_observation)

    # Termination check
    if termination_condition_check(problem, robot_id, _find_actions_count,
                                _pick_actions_count, _time_used,max_time):
        break

    if _pick_actions_count == 5 :
        print ("Pick actions count reached")
        break

In [ ]:
from pomdp_py.framework.basics import Action, Agent, POMDP, State, Observation,\
    ObservationModel, TransitionModel, GenerativeDistribution, PolicyModel,\
    sample_generative_model

In [ ]:
agent = problem.agent
tree = agent.tree
history = agent.history
state = agent.sample_belief()
action = agent.policy_model.rollout(state, history)
print (action)
next_state, observation, reward, nsteps = sample_generative_model(agent, state, action)
print (reward,nsteps)